## Einrichtung von Julia JuMP
---

Überprüfen Sie die bereits installierten Pakete. In der JuliaBox sind möglicherweise alle notwendigen Pakete bereits vorhanden.

In [1]:
using Pkg;
Pkg.status()

    Status `C:\Users\Julia\.julia\environments\v1.2\Project.toml`
  [336ed68f] CSV v0.5.16
  [a93c6f00] DataFrames v0.19.4
  [60bf3e95] GLPK v0.12.0
  [3c7084bd] GLPKMathProgInterface v0.4.4
  [7073ff75] IJulia v1.20.2
  [4076af6c] JuMP v0.20.1


Installieren Sie ``JuMP`` , mit dem Sie einfach Optimierungsprogramme formulieren können. Sowie ``GLPK`` einen open-source Solver zum Lösen der Problem und ``StatsPlots`` zur Visualisierung der Lösung.



In [2]:
#Pkg.add("JuMP")
#Pkg.add("GLPK");

Initialisieren Sie die installierten Pakete, damit sie im folgenden Notebook verwendet werden können.


In [3]:
using JuMP, GLPK;

# Planspiel Schritt 1: Aggregierte Planung

### Entscheidungsvariablen:

Bestimmung der optimalen **Produktionsmengen, Absatzmengen, Fremdbezugsmengen** und **Lagerbestände** der Produkte, sowie der optimal zu nutzenden **Zusatzkapazität** der Ressourcen je Periode, 

### Zielfunktion:

0) **Deckungsbeitragsmaximierung**: $$  \qquad \max db = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (e_iA_{it}-k_i^vX_{it}-k_i^l L_{it}-k_i^fF_{it}) - \sum_{j=1}^J \sum_{t=1}^T (k_j^zZ_{jt}) $$


### Nebenbedingungen:

1) **Kapazitaetsrestriktion:** $ \hspace{41mm} \sum_{i=1}^I(r_{ij}\cdot X_{it}) \leq c_j + z_{jt} \hspace{40mm} \forall j \in J, t \in T $

2) **Absatzobergrenze:** $\hspace{38mm} \qquad A_{it} \leq d_{it} \hspace{69mm} \forall i \in I , t \in T $


3) **Absatzuntergrenze:** $ \hspace{37mm}\qquad A_{it} \geq d_{it}^{min} \hspace{66mm} \forall i \in I , t \in T $
 
4) **Lagerbilanz:** $ \hspace{61mm} L_{it}=L_{i,t-1}+X_{it}-A_{it}+F_{it} \hspace{31mm} \forall i \in I , t \in T$

5) **Anfangslagerbestand:**   $ \hspace{43mm} L_{i,0} = l_{start} \hspace{64mm} \forall  i \in I$


6) **Max. Zusatzkapazität:** $ \hspace{44mm} Z_{jt} \leq z_j^{max}  \hspace{65mm} \forall j \in J, t \in T $

7) **Max. Fremdbezug:** $ \hspace{49mm} F_{it} \leq f_i^{max} \hspace{65mm} \forall i \in I, t \in T $

8) **Nichtnegativitätsbedingung:** $ \hspace{31mm} X_{it}, A_{it}, L_{it}, F_{it} \geq 0 \hspace{49mm} \forall i \in I, t \in T$

und $\hspace{81mm} Z_{jt} \geq 0 \hspace{71mm}  \forall j \in J, t \in T $ 

* * *

## Verwendete Symbole

### Mengen

$i \in (1,..,I)\hspace{20mm} $ Produkte  

$j \in (1,..,J)\hspace{20mm}$  Ressourcen 

$t \in (1,..,T)\hspace{20mm}$  Zeiten 

### Variablen

$X_{it}$  $\geq0$      $\hspace{28mm}$ Produktionsmenge      

$A_{it}$  $ \geq 0 $  $\hspace{28mm} $ Absatzmenge 

$L_{it}$  $\geq0$  $\hspace{28mm} $ Lagerbestand  

$F_{it}$       $ \geq0$    $\hspace{28mm} $ Fremdbezug   

$Z_{jt}$      $\geq0$      $\hspace{27mm} $ Zusatzkapazität  


### Parameter

$e_i \hspace{39mm}$   Erlös    

$k_i^v \hspace{38mm}$ variable Kosten

$k_i^l \hspace{38mm}$ Lagerkostensatz 

$k_j^z \hspace{38mm}$ Überstundenkostensatz 

$k_i^f \hspace{38mm}$ Fremdbezugskosten 

$d_{it} \hspace{38mm}$  Nachfrage  

$d_{it}^{min} \hspace{36mm}$ Absatzuntergrenze  

$r_{ij} \hspace{38mm}$  Produktionskoeffizient 

$c_j \hspace{39mm}$  Kapazität  

$l_{start} \hspace{34mm}$  Anfangslagerbestand  

$z_j^{max} \hspace{34mm}$ max. Zusatzkapazität 

$f_i^{max} \hspace{33mm}$  max. Fremdbezug

 * * *
 

## Erstellen Sie das Modell namens ``m`` und geben Sie als zu verwendenden Solver GLPK an.
---

In [4]:
m = Model(with_optimizer(GLPK.Optimizer));

### Mengen und Parameter
---

Fügen Sie die Mengen ein.

In [5]:
#Mengen
Produkte =   ["Lady", "Dude", "Rocker"];
Ressourcen = ["Herstellen"];
Perioden =   ["1", "2", "3", "4", "5", "6"];

#Längen
I = length(Produkte);
J = length(Ressourcen);
T = length(Perioden);

Fügen Sie die Parameter ein.

In [6]:
r = [  6.3/60, 5.7/60, 3.9/60]   

#Ressourcenverbrauch von Ressource j durch Produkt i

       #1,  2,    3,   4,   5,   6
d = [  1600 1400 1000 1200 1600 1400     #Lady
       1000 1200 800  1000 1600 2000     #Dude
       1200 1000 800  1400 1000 1400 ];  #Rocker

#Nachfrage von Produkt i in Periode t
 
         #1,  2,    3,   4,   5,   6
dmin = [1600 1400 1000 1200 1600 1400     #Lady
        1000 1200 800  1000 1600 2000     #Dude
        1200 1000 800  1400 1000 1400 ];  #Rocker

#Absatzuntergrenze von Produkt i in Periode t

In [7]:
c =  [320];                  #Kapazität der Ressource j

e =  [400, 540, 300];        #Erlös des Produktes

#kf = [320, 432, 240];       #Fremdbezugskostensatz 1 der Produkte
kf = [240, 324, 180];        #Fremdbezugskostensatz 2 der Produkte (günstiger)

kl = [3.2, 4.32, 2.4];       #Lagerkostensatz der Produkte

kz = [450];                  #Überstundenkostensatz von Ressource j

kv = [80, 108, 60];          #variable Herstellkosten der Produkte

zmax = [40];                 #Max. Zusatzkapazität von Ressource j

l_start =  [0, 0, 0];        #Anfangslagerbestand der Produkte


            #1,   2,   3,   4,  5,   6
fmax =  [  2000 2000 2000 2000 2000 2000   #Lady         
           2000 2000 2000 2000 2000 2000   #Dude  
           2000 2000 2000 2000 2000 2000]; #Rocker

    #Maximaler Fremdbezug von Produkt i in Periode t


### Entscheidungsvariablen
---
Definieren Sie die Entscheidungsvariablen. Achten Sie auf die Definitionsbereiche:

 **Nichtnegativitätsbedingung**: Die Entscheidungsvariablen, also die Produktions- und Absatzmengen, Lagerbestände, Fremdbezugsmengen und die in Anspruch genommenen Zusatzkapazitäten dürfen nur positive Werte annehmen, wir setzen deren Nichtnegativität voraus.

$ \qquad X_{it}, A_{it}, L_{it}, F_{it} \geq 0 \qquad \qquad \forall i \in I, t \in T$

und $\qquad \qquad Z_{jt} \geq 0 \qquad \qquad \qquad  \forall j \in J, t \in T $ 

In [8]:
@variables  m begin
            X[1:I,1:T]>=0,Int  #Produktionsmenge von Produkt i in Periode t
            A[1:I,1:T]>=0,Int  #Absatzmenge von Produkt i in Periode t
            F[1:I,1:T]>=0,Int  #Fremdbezugsmenge von Produkt i in Periode t
            Z[1:J,1:T]>=0,Int  #Zusatzkapazität von Ressource j in Periode t
            L[1:I,0:T]>=0,Int; #Lagerbestand von Produkt i am Periodenende von t
            end

## Zielfunktion

 **Deckungsbeitragsmaximierung:** Der Gesamtdeckungsbeitrag db soll maximiert werden. Dieser berechnet sich hier aus der Summe der einzelnen Produkterlöse abzüglich der variablen Kosten, abzüglich der Lagerkosten, abzüglich der Fremdbezugskosten und wird Zusätzlich noch verringert um die Zusatzkapazitätskosten der Ressourcen.

$  \qquad \max db = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (e_iA_{it}-k_i^vX_{it}-k_i^l L_{it}-k_i^fF_{it}) - \sum_{j=1}^J \sum_{t=1}^T (k_j^zZ_{jt}) $

In [9]:
@objective(m, Max,  sum(e[i] * A[i,t] - kv[i] * X[i,t] - kl[i] * L[i,t] - kf[i] * F[i,t]
                    for i=1:I for t=1:T) 
                    - sum(kz[j] *Z[j,t] for j=1:J for t=1:T));

## Nebenbedingungen

 **Kapazitätsrestriktion**: Die zur Verfügung stehende Kapazität für die auf den einzelnen Ressourcen mögliche Produktionszeit darf nicht überschritten werden. Die Summe der Gesamtproduktionszeit aller Produkte $ i = 1,...,I $ auf den einzelnen Ressourcen j darf in jeder Periode die jeweils verfügbare Kapazität  $𝑐_j$ ergänzt um die Zusatzkapazität $o_{jt}$  nicht überschreiten.

$ \qquad \sum_{i=1}^I(r_{ij}\cdot X_{it}) \leq c_j + z_{jt} \qquad \forall j \in J, t \in T $

In [10]:
@constraint(m, KapRes[t=1:T,j=1:J], sum(r[i,j] * X[i,t] for i=1:I) <= c[j] + Z[j,t]);

**Absatzuntergrenze**: Für die Produkte existieren Absatzuntergrenzen basierend auf der Nachfrage. Die abgesetzte Menge der einzelnen Produkte kann diese Absatzuntergrenze in keiner Periode unterschreiten.

$ \qquad A_{it} \geq d_{it}^{min} \qquad \qquad \qquad \qquad \forall i \in I , t \in T $

In [11]:
@constraint(m, AbsUn[i=1:I,t=1:T], A[i,t] >= dmin[i,t]);

**Absatzobergrenze**: Für die Produkte existieren Absatzober- und untergrenzen basierend auf der Nachfrage. Die abgesetzte Menge der einzelnen Produkte kann diese Absatzobergrenze in keiner Periode überschreiten.

$ \qquad A_{it} \leq d_{it}^{max} \qquad \qquad \qquad \qquad \forall i \in I , t \in T $

In [12]:
@constraint(m, AbsOb[i=1:I,t=1:T], A[i,t] <= d[i,t]);

 **Lagerbilanz**: Der Lagerbestand eines Produktes am Ende einer Periode berechnet sich aus der Summe der eingelagerten Menge in der Vorperiode, der Produktionsmenge und der Fremdbezugsmenge, abzüglich der abgesetzen Menge des Produktes.

$ \qquad L_{it}=L_{i,t-1}+X_{it}-A_{it}+F_{it} \qquad \forall i \in I , t \in T$

In [13]:
@constraint(m, Lager[i=1:I,t=1:T], L[i,t] == L[i,t-1] + X[i,t] + F[i,t] - A[i,t]);

 **Anfangslagerbestand**: Der Anfangslagerbestand aller Produkte entspricht dem initial gesetzen $l_i$.
 
  $$ \qquad L_{i,0} = l_{start} \hspace{40mm} \forall  i \in I$$


In [14]:
@constraint(m, AnfLager[i=1:I], L[i,0] == l_start[i]);

 **Max. Zusatzkapazität**: Die eingesetzten Zusatzkapazitäten (Überstunden) für jede Ressource müssen in jeder Periode unter der Obergrenze, der maximalen Zusatzkapazität, bleiben. Die maximale Zusatzkapazität ist für jede Ressource j einzeln gegeben.

$ \qquad Z_{jt} \leq z_j^{max}  \qquad \qquad \qquad \qquad \forall j \in J, t \in T $

In [15]:
@constraint(m, ZusatzKapa[j=1:J,t=1:T], Z[j,t] <= zmax[j] );

**Max. Fremdbezug**: Der dazugekaufte Fremdbezug jedes Produktes muss in allen Perioden unter der Obergrenze, dem maximalen Fremdbezug, bleiben. Der maximale Fremdbezug ist für jedes Produkt i einzeln gegeben. 

$ \qquad F_{it} \leq f_i^{max} \qquad \qquad \qquad \qquad \forall i \in I, t \in T $

In [16]:
@constraint(m, FremdB[i=1:I,t=1:T], F[i,t] <= fmax[i,t] );

## Lösen Sie das Modell.
---

In [17]:
optimize!(m)

Lassen Sie sich das Modell anzeigen.

In [18]:
m

A JuMP Model
Maximization problem with:
Variables: 81
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 21 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 18 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 48 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 81 constraints
`VariableRef`-in-`MathOptInterface.Integer`: 81 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: A, AbsOb, AbsUn, AnfLager, F, FremdB, KapRes, L, Lager, X, Z, ZusatzKapa

Lassen Sie sich den Zielfunktionswert anzeigen.

In [32]:
db= JuMP.objective_value(m)
println("Objective value db: ", round(Int64,db))

Objective value db: 7484324


In [21]:
JuMP.value.(Z)

1×6 Array{Float64,2}:
 21.0  6.0  0.0  0.0  38.0  40.0

In [22]:
JuMP.value.(L)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:3
    Dimension 2, 0:6
And data, a 3×7 Array{Float64,2}:
 0.0  0.0  0.0  823.0  902.0  647.0  0.0
 0.0  0.0  0.0    0.0    1.0    0.0  0.0
 0.0  0.0  0.0    9.0    3.0    1.0  0.0

In [23]:
JuMP.value.(F)

3×6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [24]:
JuMP.value.(A)

3×6 Array{Float64,2}:
 1600.0  1400.0  1000.0  1200.0  1600.0  1400.0
 1000.0  1200.0   800.0  1000.0  1600.0  2000.0
 1200.0  1000.0   800.0  1400.0  1000.0  1400.0

In [25]:
JuMP.value.(X)

3×6 Array{Float64,2}:
 1600.0  1400.0  1823.0  1279.0  1345.0   753.0
 1000.0  1200.0   800.0  1001.0  1599.0  2000.0
 1200.0  1000.0   809.0  1394.0   998.0  1399.0

In [26]:
sum(JuMP.value.(A[2,t])*e[2] for t in 1:T)

4.104e6

In [27]:
println(round(Int64,2.04e6))

2040000
